In [ ]:
import os
# os.listdir('/mnt/c/Users/Mukul/Documents/docker files/ee981')

In [ ]:
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import  Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, concatenate, Input, BatchNormalization, Conv2DTranspose
import sklearn as skl
import matplotlib.pyplot as plt
import cv2

# print("Tensorflow version " + tf.__version__)

# -- TPU code
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# print('Running on TPU ', tpu.cluster_spec().as_dict())

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# strategy = tf.distribute.experimental.TPUStrategy(tpu)
# print("REPLICAS: ", strategy.num_replicas_in_sync)


# 0.5 - Hyperparameters


In [ ]:
BATCH_SIZE = 16
TEST_SPLIT = 0.10
EPOCHS = 30
LEARNING_RATE = 0.001
IMAGE_SIZE = (2160,3840,3) # 3840 or 3480?
TARGET_SIZE = (512 ,512,3)

# 1 - Preprocessing

Due to inconsistent documentation it was unclear if the functions were being resolved eagerly or lazily. ~~It was universally reccomended that the tf.function decorator enabled lazy processing though.~~ Using the tf.function decorator caused the kernel to crash.


In [ ]:

# RGB_FIRE_PATH  = '/content/drive/MyDrive/Colab Notebooks/Colab files/EE981/Images/'
# MASK_FIRE_PATH = '/content/drive/MyDrive/Colab Notebooks/Colab files/EE981/Masks/'
RGB_FIRE_PATH  = '/mnt/c/Users/Mukul/Documents/docker files/ee981/EE981/Images/'
MASK_FIRE_PATH = '/mnt/c/Users/Mukul/Documents/docker files/ee981/EE981/Masks/'

img_paths = [RGB_FIRE_PATH + 'image_' + str(i) + '.jpg' for i in range(2002)]
mask_paths  = [MASK_FIRE_PATH + 'image_' + str(i) + '.png' for i in range(2002)]



In [ ]:
# @tf.function # Converts following to polymorphic func
def get_image(filename):
    """ Lazily reads and resizes images
    Args:
        filename (str): String path. location of image
    Returns:
        (tf.tensor , tf.tensor): returns decoded image and masks of size TARGET_SIZE
    """
    img = tf.io.read_file(filename[0])
    mask = tf.io.read_file(filename[1])
    img = tf.image.decode_jpeg(img, channels=3)
    mask = tf.image.decode_png(mask)
    img = tf.image.resize(img,(TARGET_SIZE[0],TARGET_SIZE[1]),method ='nearest')
    mask = tf.image.resize(mask,(TARGET_SIZE[0],TARGET_SIZE[1]), method = 'nearest')
    return (img, mask)

# @tf.function
def prepare_ds(x, y):
    """ Maps the get_image function to the required img dataset and batches
    Args:
        x (list(str)): list of img paths
        y (list(str)): list of mask paths

    Returns:
        tf.dataset: batched img dataset
    """
    zipped_paths = list(zip(x,y))
    img_dataset = tf.data.Dataset.from_tensor_slices(zipped_paths)
    img_dataset = img_dataset.map(get_image)
    img_dataset = img_dataset.batch(BATCH_SIZE)
    
    return img_dataset
    
train_paths_x, tmp_x, train_paths_y, tmp_y  = skl.model_selection.train_test_split(img_paths, mask_paths, test_size=TEST_SPLIT*2) # shuffles
test_paths_x, val_paths_x, test_paths_y, val_paths_y  = skl.model_selection.train_test_split(tmp_x, tmp_y, test_size=0.5) # shuffles
print(f'length of train: {len(train_paths_x)}, val: {len(val_paths_x)}, test: {len(test_paths_x)}')

train_ds = prepare_ds(train_paths_x, train_paths_y)
val_ds = prepare_ds(val_paths_x, val_paths_y)
test_ds = prepare_ds(test_paths_x, test_paths_y)


# 2 - Constructing the U-Net Architecture

## 2.05 - U-Net from FLAME

This model is the same one from the FLAME paper. 

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout, Lambda
from tensorflow.keras.layers import Conv2D, Conv2DTranspose

def model_unet_kaggle(input_size=TARGET_SIZE, num_classes=2):
    """
    This function returns a U-Net Model for this binary fire segmentation images:
    Arxiv Link for U-Net: https://arxiv.org/abs/1505.04597
    :param img_hieght: Image Height
    :param img_width: Image Width
    :param img_channel: Number of channels in each image
    :param num_classes: Number of classes based on the Ground Truth Masks
    :return: A convolutional NN based on Tensorflow and Keras
    """
    inputs = Input(input_size)
    s = Lambda(lambda x: x / 255)(inputs)

    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# 3 - Train / Load model


In [ ]:
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.ops import clip,log

class wbce(tf.keras.Loss):
    def __init__(self, rate=1e-2):
        super().__init__()
        self.rate = rate

    def get_config(self):
        base_config = super().get_config()
        return base_config
        
    def call(self,y_true, y_pred):
        """This is the custom WBCE function called during eval.
        Applies a weight map to the BCE loss.

        Args:
            y_true: True img
            y_pred: Predicted img

        Returns:
           float64: loss metric
        """
        tf_y_true = tf.cast(y_true, dtype=y_pred.dtype)
        tf_y_pred = tf.cast(y_pred, dtype=y_pred.dtype)

        weights_v = tf.where(tf.equal(tf_y_true, 1),0.90, 0.10)
        weights_v = tf.cast(weights_v, dtype=y_pred.dtype)
        ce = tf.keras.losses.binary_crossentropy(tf_y_true, tf_y_pred, from_logits=False)
        # print(ce,(weights_v))
        loss = tf.keras.ops.mean(tf.multiply(ce, tf.squeeze(weights_v)))
        return loss


In [ ]:
def train_model():
    """
        Compiles and trains the model using two callbacks
        Returns:
           Keras.model: trained model object
           Keras.History: model training history object      
    """
           
    model = model_unet_kaggle()
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=LEARNING_RATE),
                    loss=wbce, #'binary_crossentropy',
                    metrics=[ tf.keras.metrics.Precision(),
                            tf.keras.metrics.Recall(),
                            tf.keras.metrics.MeanIoU(2),
                            tf.keras.metrics.IoU(num_classes = 2,target_class_ids=[0,1]),
                            'accuracy'])
    
    scheduler = lambda epoch: LEARNING_RATE if epoch<5 else LEARNING_RATE/100
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
    
    history = model.fit(train_ds, validation_data= val_ds,
                        epochs = EPOCHS,
                        callbacks = [
                            early_stop,
                            tf.keras.callbacks.LearningRateScheduler(scheduler)
                            ]
                        )
    return model, history

def load_model(path):
    return tf.keras.models.load_model(path)
    
# model = load_model(os.path.join('../working_unet_720_720.keras'))
model, history = train_model()

# Save/Load model

In [ ]:
import pickle

def save_model(model, history, name ):
    model.save_weights(name + '.weights.h5')
    with open(name + '.history','wb') as f:
        pickle.dump(history.history,f)

def load_model(model_name):
    model = model_unet_kaggle()
    model.load_weights(model_name + '.weights.h5')#custom_objects={ 'loss':wbce },compile = False)
    with open(model_name + '.history', 'rb') as f:
        history = pickle.load(f)
    return model,history
# model,history = load_model('paper_unet_512_512_norm_wbce')
# save_model(model,history,'paper_unet_512_512_norm_wbce')

# 4 - Plot and evaluate

In [ ]:
input_img_it =  test_ds

# in, gt = ds.take(4)[0]
for i in test_ds.take(4):
    input_img = i[0]
    input_gt = i[1]
    
mask = model.predict(np.array([input_img[0]]))

fig,((ax1,ax2,),(ax3,ax4,)) = plt.subplots(2,2,figsize=(8,8))
fig.suptitle('Weighted Binary Cross entropy')

ax1.set_title('Input image')
ax1.imshow(np.array(input_img[0]).astype(np.uint8))

ax2.set_title('Ground truth')
ax2.imshow(input_gt[0])
print(np.unique(input_gt[0]))
ax3.set_title('Model output - No thresh')
ax3.imshow(mask[0])

ax4.set_title('Thresholded output')
threshed_mask = np.where(mask[0] < 0.9, 0, 1)
ax4.imshow(threshed_mask)

# plt.savefig('weighted_BCE-fig.png')


In [ ]:
# -- precision recall

p_r_vals = []
thresh_vals = range(1,9,1)

def recall_m(y_true, y_pred):
    """Calculate recall metrics

    Args:
      y_true (tf.tensor): ground truth
      y_pred (tf.tensor): prediction from model

    Returns:
        np.float32: recall value
    """
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = tf.keras.ops.sum(y_true * y_pred)
    possible_positives = tf.keras.ops.sum(y_true)
    recall = true_positives / (possible_positives +tf.keras.backend.epsilon())
    return recall

def precision_m(y_true, y_pred):
    """Calculate precision metrics

    Args:
      y_true (tf.tensor): ground truth
      y_pred (tf.tensor): prediction from model
    
    Returns:
        np.float32: precision value
    """
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    true_positives = tf.keras.ops.sum(y_true * y_pred)
    all_positives = tf.keras.ops.sum(y_pred)

    recall = true_positives / (all_positives +tf.keras.backend.epsilon())
    return recall



In [ ]:
model.evaluate(test_ds)

In [ ]:

# Generates precision + recall for different thresholds
r_p=[]
for thresh in thresh_vals:

  threshed_mask = np.where(mask[0] <= thresh/10, 0, 1)
  r_p.append((tf.keras.backend.eval(recall_m(input_gt[0],threshed_mask)),
  tf.keras.backend.eval(precision_m(input_gt[0],threshed_mask))))
  plt.annotate(f'$0.{thresh}$',(r_p[-1][0],r_p[-1][1]))

plt.plot(*zip(*r_p))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')

In [ ]:
import pickle
# with open('/content/drive/MyDrive/Colab Notebooks/Colab files/paper_unet_512_512_norm_wbce.history','rb') as f:
#     history = pickle.load(f)
# history = history.history
print(history.keys())

# Unsure why but the subscript next to the metric keeps changing. when generating plots the subscript needs to be changed to match history.history dict
plt.figure(figsize=(12,4))
ax = plt.subplot(1,2,1)
plt.plot(history['recall_1'], color='blue', label = 'train recall')
plt.plot(history['val_recall_1'], linestyle='dashed',color='blue',label = 'val recall')
ax.set_title('Training vs Validation performance')
plt.xlabel('Epochs')
plt.ylabel('percentage')
plt.plot(history['val_precision_1'],  linestyle='dashed',color='orange',label = 'val precision')
plt.plot(history['precision_1'],color='orange',label = 'train precision')
plt.legend()
# plt.set_title('precision')

ax = plt.subplot(1,2,2)
plt.plot(history['loss'],color= 'blue', label = 'Training loss')
plt.plot(history['val_loss'],color = 'orange',label='Validation loss')
plt.legend()
ax.set_title('weighted binary cross entropy loss')
plt.figure()

# Load and predict video

In [ ]:
import  cv2
import timeit
def process_vid_frame(frame):   
    return tf.image.resize(frame,(TARGET_SIZE[0],TARGET_SIZE[1]))

vid = cv2.VideoCapture('1-Zenmuse_X4S_1.mp4')
width  = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))   # float `width`
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))  # float `height`
fps = vid.get(cv2.CAP_PROP_FPS)
mask_vid = cv2.VideoWriter(filename='mask.mp4',fourcc=cv2.VideoWriter_fourcc(*'mp4v'),
                           fps = fps,frameSize = (512,512),isColor=False) # writes mask video
main_vid = cv2.VideoWriter(filename='vid.mp4',fourcc=cv2.VideoWriter_fourcc(*'mp4v'), 
                           fps = fps,frameSize = (512,512),isColor=True) # outputs the frames selected for processing

# Streaming has some overhead
i=0
m_ar=[]
tmp_ar=[]
frames_to_skip = 0

# This while loop also generates an adaditional video consisting only of  
# frames from the input that were used for the prediction (used for the presentation)
while (vid.isOpened() and i<5000):
    
    ret, frame = vid.read()
    print(i) if i%100 == 0 else None
    
    if i % (frames_to_skip + 1) == 0:
        
        input_img = process_vid_frame(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        mask_prediction = model.predict(input_img[None,...])
        mask_prediction = np.where(mask_prediction < 0.3, 0, 255)
        m_ar.append(input_img)
        tmp_ar.append(mask_prediction)
        main_vid.write( tf.keras.backend.eval(input_img)[...,::-1].astype(np.uint8))
        mask_vid.write(np.squeeze(mask_prediction).astype(np.uint8))
    
    i+=1

vid.release()
mask_vid.release()
main_vid.release()